## **New Dynamic Code**:
15-April, 2024

In [1]:
# Install pyod
!pip install pyod -q

import os
from datetime import datetime
import numpy as np
import pandas as pd

from scipy.stats import ttest_ind
from sklearn.preprocessing import MinMaxScaler

# Import pyod modules one by one
from pyod.models.pca import PCA
from pyod.models.ecod import ECOD
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
# from pyod.models.knn import KNN
from pyod.models.copod import COPOD
from pyod.models.gmm import GMM
from pyod.models.loda import LODA
from pyod.models.iforest import IForest

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

#### Helper functions

In [2]:
def combine_dataframes(dataframes):
    # Concatenate the dataframes into a single dataframe
    df_combined = pd.concat(dataframes, axis=1)
    
    # Generate column names based on a pattern
    df_combined.columns = [dfs[i].columns[0] for i in range(len(dataframes))]

    # Remove rows with NaN values
    df_combined = df_combined.dropna()

    # Add a new column named 'time'
    number_points = len(df_combined)
    dt = 1 / 2999
    start = 0
    points = [start + i * dt for i in range(number_points)]
    df_combined['time'] = points
    
    return df_combined

def split_groups(df):
    # Extract the rows for the first group (0 to 300)
    group1 = df[df['time']<=300]

    # Extract the rows for the second group (1500 to 1800)
    group2 = df[(df['time'] >= 1500) & (df['time'] <= 1800)]
    
    return group1, group2

# List of auxiliary functions
def fit_and_get_scores(model, data):
    model.fit(data)
    decision_scores = model.decision_scores_
    return model, decision_scores

def scale_and_average_scores(scores_df):
    # Set random seed for reproducibility
    np.random.seed(42)
    scaler = MinMaxScaler()
    scaled_scores = scaler.fit_transform(scores_df[['PCA', 'ECOD', 'CBLOF', 'HBOS',  'COPOD', 'GMM', 'LODA', 'IForest']])
    scores_df[['PCA', 'ECOD', 'CBLOF', 'HBOS',  'COPOD', 'GMM', 'LODA', 'IForest' ]] = scaled_scores
    scores_df['Average_Score'] = scores_df[['PCA', 'ECOD', 'CBLOF', 'HBOS',  'COPOD', 'GMM', 'LODA', 'IForest']].mean(axis=1)
    return scores_df

def display_and_style_df(df, title):
    # Reset the index to display correctly
    df.reset_index(drop=True, inplace=True)

    # Display DataFrame with both data and styled caption
    display(pd.DataFrame(df).style.set_caption(title).format("{:.2f}"))

def process_group_scores(group_df ):
    RANDOM_SEED = 42
    np.random.seed(RANDOM_SEED)
    
    # Create instances of PyOD models
    models = [
        PCA(),
        ECOD(),
        CBLOF(n_jobs=-1),
        HBOS(),
        # KNN(n_jobs=-1),
        COPOD(),
        GMM(),
        LODA(),
        IForest(n_jobs=-1)
    ]

    # Set random state for models that support it
    for model in models:
        if hasattr(model, 'random_state'):
            model.random_state = RANDOM_SEED
    
    
    scores_before_scale = pd.DataFrame()

    for model in models:
        model_name = model.__class__.__name__
        print(model_name)
        _, scores = fit_and_get_scores(model, group_df.values.reshape(-1, 1))
        scores_before_scale[model_name] = scores

    scores_before_scale['Average_Score'] = scores_before_scale.mean(axis=1)

    # Concatenate the toy data DataFrame with the scores DataFrame and display it
    display_df_before_scale = pd.concat([group_df, scores_before_scale], axis=1)

    scores_after_scale = scale_and_average_scores(scores_before_scale.copy())

    # Concatenate the toy data DataFrame with the scaled scores DataFrame and display it
    display_df_after_scale = pd.concat([group_df, scores_after_scale], axis=1)

    return display_df_before_scale, display_df_after_scale


def stack_columns_except_time(df):
    stacked_df = pd.concat([df[column] for column in df.columns if column != 'time'], ignore_index=True)
    return stacked_df

def before_after_split(dfs):
    wr_df = combine_dataframes(dfs)
    wr_df.sort_index(axis=1, inplace=True)
    print('\n\n--- DF | Head & Tail---\n\n')
    print('Shapes: ',wr_df.shape)
    display(wr_df.head(2),wr_df.tail(2))
    
    
    wr_group1, wr_group2 = split_groups(wr_df)
    print('\n\n--- Group 1 | Groupd 2 | Tail---\n\n')
    print('Shapes: ',wr_group1.shape, wr_group2.shape)
    display(wr_group1.tail(2),wr_group2.tail(2))
    
    stack_before_5_min = stack_columns_except_time(wr_group1)
    stack_after_30_min = stack_columns_except_time(wr_group2)
    print('\n\n--- 5 Min | 30 Min | Tail---\n\n')
    print('Shapes: ',stack_before_5_min.shape, stack_after_30_min.shape)
    display(stack_before_5_min.tail(2),' ',stack_after_30_min.tail(2))
    
    return stack_before_5_min, stack_after_30_min
    


In [3]:
main_path = '/kaggle/input/zara-analysis-wr/zara-analysis/'
!ls /kaggle/input/zara-analysis-wr/zara-analysis

CH1-wr	CH2-wr	CH3-wr	CH4-wr


# Figure i)

In [4]:
import os

os.makedirs('/kaggle/working/fig1/')
for folder in ['CH1-wr','CH2-wr','CH3-wr','CH4-wr']:
    os.makedirs('/kaggle/working/fig1/'+folder)

In [5]:
# os.makedirs('/kaggle/working/fig1/')

for folder in ['CH1-wr','CH2-wr','CH3-wr','CH4-wr']:
    folder_path = main_path + folder

    # Get a list of all text files in the folder
    text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    print(folder)
    print(text_files)
    # Initialize an empty numpy array to store the dataframes
    dfs = []

    # Iterate over the text files and read their contents into dataframes
    for i, file in enumerate(text_files):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, sep='\t', header=None, names=[str(file)])
        dfs.append(df)
        
    stack_before_5_min, stack_after_30_min = before_after_split(dfs)
#     os.makedirs('/kaggle/working/fig1/'+folder)
    
    print('Before 5 min')
    print("Current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    before_scale_5_min, after_scale_5_min = process_group_scores(stack_before_5_min)
    
    before_scale_5_min.to_csv('/kaggle/working/fig1/'+folder+'/before_scale_5_min.csv')
    after_scale_5_min.to_csv('/kaggle/working/fig1/'+folder+'/after_scale_5_min.csv')

    print('After 30 min')
    print("Current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    before_scale_30_min, after_scale_30_min = process_group_scores(stack_after_30_min)
    
    before_scale_30_min.to_csv('/kaggle/working/fig1/'+folder+'/before_scale_30_min.csv')
    after_scale_30_min.to_csv('/kaggle/working/fig1/'+folder+'/after_scale_30_min.csv')
    

CH1-wr
['WR17Ch1.txt', 'WR20Ch1.txt', 'WR16Ch1.txt', 'WR18Ch1.txt', 'WR19Ch1.txt']


--- DF | Head & Tail---


Shapes:  (6327228, 6)


,WR16Ch1.txt,WR17Ch1.txt,WR18Ch1.txt,WR19Ch1.txt,WR20Ch1.txt,time
0,258.4224,280.8238,268.0998,267.5622,272.7593,0.000000
1,252.6877,277.4188,265.4117,272.2217,271.5048,0.000333


,WR16Ch1.txt,WR17Ch1.txt,WR18Ch1.txt,WR19Ch1.txt,WR20Ch1.txt,time
6327226,270.6088,258.4224,270.4296,272.5801,273.1177,2109.778593
6327227,265.4117,259.8561,266.6661,270.2504,265.0532,2109.778926




--- Group 1 | Groupd 2 | Tail---


Shapes:  (899701, 6) (899701, 6)


,WR16Ch1.txt,WR17Ch1.txt,WR18Ch1.txt,WR19Ch1.txt,WR20Ch1.txt,time
899699,272.9385,271.5048,274.3722,270.788,267.3830,299.999667
899700,265.2324,272.5801,268.9959,260.573,263.6195,300.000000


...

Before 5 min
Current time: 2024-04-15 20:24:29
PCA
ECOD
CBLOF


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


HBOS
COPOD
GMM
LODA
IForest
After 30 min
Current time: 2024-04-15 20:35:10
PCA
ECOD
CBLOF


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


HBOS
COPOD
GMM
LODA
IForest


In [6]:
1

1